In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import sparse
import networkx as nx
import time
from datetime import date
import random
import scipy
import seaborn as sns
from scipy.sparse.csgraph import shortest_path
import json
import pickle
from sklearn.utils import shuffle
import os
np.random.seed(42)
random.seed(42)

path = '../input/scisci-data/' #change this according to your path

NUM_OF_VERTICES=64719 # number of vertices of the semantic net

data_source = path+'CompetitionSet2017_3.pkl'
#data_source=path+'TrainSet2014_3.pkl'
full_dynamic_graph_sparse,unconnected_vertex_pairs,year_start,years_delta = pickle.load( open( data_source, "rb" ) )

with open(path+'TrainSet2014_3_solution.pkl', "rb" ) as pkl_file:
        unconnected_vertex_pairs_solution = pickle.load(pkl_file)
        

print(data_source+' has '+str(len(full_dynamic_graph_sparse))+' edges between a total of '+str(NUM_OF_VERTICES)+ ' vertices.\n\n')
print('The goal is to predict which of '+str(len(unconnected_vertex_pairs))+' unconnectedvertex-pairs\nin unconnected_vertex_pairs will be connected until '+str(year_start+years_delta)+'.')

../input/scisci-data/CompetitionSet2017_3.pkl has 7652945 edges between a total of 64719 vertices.


The goal is to predict which of 1000000 unconnectedvertex-pairs
in unconnected_vertex_pairs will be connected until 2020.


In [2]:
unconnected_vertex_pairs_solution[unconnected_vertex_pairs_solution == 1].shape

(492,)

## Construyendo el grafo

In [3]:

def create_training_data(full_graph,year_start,years_delta,edges_used=10000,vertex_degree_cutoff=10):
    """
    :param full_graph: Full graph, numpy array dim(n,3) [vertex 1, vertex 2, time stamp]
    :param year_start: year of graph
    :param years_delta: distance for prediction in years (prediction on graph of year_start+years_delta)
    :param edges_used: optional filter to create a random subset of edges for rapid prototyping (default: 500,000)
    :param vertex_degree_cutoff: optional filter, for vertices in training set having a minimal degree of at least vertex_degree_cutoff  (default: 10)
    :return:

    all_edge_list: graph of year_start, numpy array dim(n,3)
    unconnected_vertex_pairs: potential edges for year_start+years_delta
    unconnected_vertex_pairs_solution: numpy array with integers (0=unconnected, 1=connected), solution, length = len(unconnected_vertex_pairs)
    """

    years=[year_start,year_start+years_delta]    #2011 - 2014
    day_origin = date(1990,1,1)

    all_G=[]
    all_edge_lists=[]
    all_sparse=[]
    for yy in years:
        print('    Create Graph for ', yy)
        day_curr=date(yy,12,31)
        all_edges_curr=full_graph[full_graph[:,2]<(day_curr-day_origin).days]
        adj_mat_sparse_curr = sparse.csr_matrix((np.ones(len(all_edges_curr)), 
                                                 (all_edges_curr[:,0], all_edges_curr[:,1])),
                                                shape=(NUM_OF_VERTICES,NUM_OF_VERTICES))
        G_curr=nx.from_scipy_sparse_matrix(adj_mat_sparse_curr, 
                                           parallel_edges=False, create_using=None, 
                                           edge_attribute='weight')

        all_G.append(G_curr)
        all_sparse.append(adj_mat_sparse_curr)
        all_edge_lists.append(all_edges_curr)

        print('    Done: Create Graph for ', yy)
        print('    num of edges: ', G_curr.number_of_edges())

    all_degs=np.array(all_sparse[0].sum(0))[0]

    ## Create all edges to be predicted
    all_vertices=np.array(range(NUM_OF_VERTICES))
    vertex_large_degs=all_vertices[all_degs>=1] # use only vertices with degrees larger than 10.

    unconnected_vertex_pairs=[]
    unconnected_vertex_pairs_solution=[]

    time_start=time.time()
    # agregar aristas con target positivo
    day_curr=date(years[0],12,31)
    day_ult = date(years[-1],12,31)
    all_edges_positives = all_edges_curr[(all_edges_curr[:,2]>(day_curr-day_origin).days)
                                         &(all_edges_curr[:,2] < (day_ult-day_origin).days)]
    
    all_edges_positives = all_edges_positives[np.random.choice(np.array(range(len(all_edges_positives))),
                                           6000,replace=False)] # se termina dividiendo por 2 cuando se eliminan nodos con degree<10
    
    unconnected_vertex_pairs = []
    for elem in all_edges_positives:
        if(elem[0] in vertex_large_degs) or (elem[1] in vertex_large_degs):
            if(not all_G[0].has_edge(elem[0],elem[1])):
                if (((elem[0],elem[1]) not in unconnected_vertex_pairs) 
                    and ((elem[1],elem[0]) not in unconnected_vertex_pairs)):
                    unconnected_vertex_pairs.append((elem[0],elem[1]))
    
    unconnected_vertex_pairs_solution = [1 for elem in unconnected_vertex_pairs]

    
    while len(unconnected_vertex_pairs)<edges_used:        
        v1,v2=random.sample(range(len(all_vertices)), 2)

        if (v1!=v2) and (not all_G[0].has_edge(v1,v2)):
            if(v1 in vertex_large_degs) or (v2 in vertex_large_degs):
                if len(unconnected_vertex_pairs)%10**6==0:
                    time_end=time.time()
                    print('    edge progress (',time_end-time_start,'sec): ',len(unconnected_vertex_pairs)/10**6,'M/',edges_used/10**6,'M')
                    time_start=time.time()
                unconnected_vertex_pairs.append((v1,v2))
                unconnected_vertex_pairs_solution.append(all_G[1].has_edge(v1,v2))

        
    print('Number of unconnected vertex pairs for prediction: ', len(unconnected_vertex_pairs_solution))
    print('Number of vertex pairs that will be connected: ' , sum(unconnected_vertex_pairs_solution))
    print('Ratio of vertex pairs that will be connected: ' , sum(unconnected_vertex_pairs_solution)/len(unconnected_vertex_pairs_solution))
    
    unconnected_vertex_pairs=np.array(unconnected_vertex_pairs)
    unconnected_vertex_pairs_solution=np.array(list(map(int, unconnected_vertex_pairs_solution)))
    
    
    all_edge_list=np.array(all_edge_lists[0]) #todas aristas del año input
    
    return all_edge_list, unconnected_vertex_pairs, unconnected_vertex_pairs_solution


vertex_degree_cutoff=1
train_dynamic_graph_sparse,train_edges_for_checking,train_edges_solution = (create_training_data(full_dynamic_graph_sparse, 
                                                                         year_start-years_delta, years_delta, 
                                                                             vertex_degree_cutoff=vertex_degree_cutoff))


adj_mat_sparse_curr_train = sparse.csr_matrix((np.ones(len(train_dynamic_graph_sparse)), 
                                               (train_dynamic_graph_sparse[:,0], train_dynamic_graph_sparse[:,1])), 
                                              shape=(NUM_OF_VERTICES,NUM_OF_VERTICES))

train_edges_for_checking,train_edges_solution = shuffle(train_edges_for_checking,
                                                        train_edges_solution,
                                                        random_state=8)


#para testear
adj_mat_sparse_curr_test = sparse.csr_matrix((np.ones(len(full_dynamic_graph_sparse[:,2])), 
                                               (full_dynamic_graph_sparse[:,0], full_dynamic_graph_sparse[:,1])), 
                                              shape=(NUM_OF_VERTICES,NUM_OF_VERTICES))



    Create Graph for  2014
    Done: Create Graph for  2014
    num of edges:  1843253
    Create Graph for  2017
    Done: Create Graph for  2017
    num of edges:  5568240
Number of unconnected vertex pairs for prediction:  10000
Number of vertex pairs that will be connected:  4692
Ratio of vertex pairs that will be connected:  0.4692


## Construir Subgrafos locales
current = 100.000, 1-hop

In [4]:
def neighbors(fringe, A):
    res = set(A[list(fringe)].indices)
    return res


In [5]:
import torch
def drnl_node_labeling(adj, src, dst):
    # Double Radius Node Labeling (DRNL).
    src, dst = (dst, src) if src > dst else (src, dst)

    idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
    adj_wo_src = adj[idx, :][:, idx]

    idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
    adj_wo_dst = adj[idx, :][:, idx]

    dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src)
    dist2src = np.insert(dist2src, dst, 0, axis=0)
    dist2src = torch.from_numpy(dist2src)

    dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst-1)
    dist2dst = np.insert(dist2dst, src, 0, axis=0)
    dist2dst = torch.from_numpy(dist2dst)

    dist = dist2src + dist2dst
    dist_over_2, dist_mod_2 = dist // 2, dist % 2

    z = 1 + torch.min(dist2src, dist2dst)
    z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
    z[src] = 1.
    z[dst] = 1.
    z[torch.isnan(z)] = 0.
    z = z.numpy()
    z[z>9] = 9

    return z

In [6]:

%env JOBLIB_TEMP_FOLDER=/tmp
def k_hop_subgraph(src, dst, num_hops, A,target,
                   max_nodes=None,sample_ratio=0.35):

    nodes = [src, dst]
    visited = set([src, dst])
    fringe = set([src, dst])
    for dist in range(1, num_hops+1):
        fringe = neighbors(fringe, A)
        fringe = fringe - visited
        visited = visited.union(fringe)
        if sample_ratio < 1.0:
            fringe = random.sample(fringe, int(sample_ratio*len(fringe)))
        if max_nodes is not None:
            if max_nodes < len(fringe):
                fringe = random.sample(fringe, max_nodes)
        if len(fringe) == 0:
            break
        nodes = nodes + list(fringe)
    
    nodes = np.array(nodes)
    subgraph = A[nodes, :][:, nodes]

    if len(nodes) == 2: # si no hay degree
        if target == 1:
            target = -1
        if target == 0:
            target = -2
        return subgraph,np.array([1,1]).reshape(-1,1),target
    
    
    node_features = drnl_node_labeling(subgraph.copy(), 0, 1)
    
    return subgraph,node_features,target

env: JOBLIB_TEMP_FOLDER=/tmp


In [7]:
os.mkdir("train")
lista = []
print("Generando datos de entrenamiento")
for cont,arista in enumerate(train_edges_for_checking):
    A,X,target = k_hop_subgraph(src=arista[0], dst=arista[1],
                                num_hops=1,A=adj_mat_sparse_curr_train,
                                target=int(train_edges_solution[cont]))
    if target < 0:
        target = target+2
    lista.append(target)
    scipy.sparse.save_npz('train/'+str(cont)+"_adj_V1_train", A, compressed=True)
    np.save('train/'+str(cont)+"_x_V1_train", X)


    if cont%1000 == 0:
        print(str(100*cont/len(train_edges_for_checking))+'%')

!tar -zcf train.tar.gz /kaggle/working/train/
!rm -R train        

lista = np.array(lista)
np.save('targets_train',lista)
print("listo")
print("Generando datos de test")
os.mkdir("0_test")
cont2 = 0
lista = []
for cont,arista in enumerate(unconnected_vertex_pairs):
    A,X,target = k_hop_subgraph(src=arista[0], dst=arista[1],
                num_hops=1,A=adj_mat_sparse_curr_test,
                target = unconnected_vertex_pairs_solution[cont])
    
    lista.append(target)
    if target >= 0:
        scipy.sparse.save_npz(str(cont2)+'_test/'+str(cont)+"_adj_V1_test", A, compressed=True)
        np.save(str(cont2)+'_test/'+str(cont)+"_x_V1_test", X)
        
    if cont%100000 == 0:
        if cont2 == 0:
            !tar -zcf cero.tar.gz /kaggle/working/0_test/
            !rm -R 0_test
        if cont2 == 1:
            !tar -zcf uno.tar.gz /kaggle/working/1_test/
            !rm -R 1_test
        if cont2 == 2:
            !tar -zcf dos.tar.gz /kaggle/working/2_test/
            !rm -R 2_test
        if cont2 == 3:
            !tar -zcf tres.tar.gz /kaggle/working/3_test/
            !rm -R 3_test
        if cont2 == 4:
            !tar -zcf cuatro.tar.gz /kaggle/working/4_test/
            !rm -R 4_test
        if cont2 == 5:
            !tar -zcf cinco.tar.gz /kaggle/working/5_test/
            !rm -R 5_test
        if cont2 == 6:
            !tar -zcf seis.tar.gz /kaggle/working/6_test/
            !rm -R 6_test
        if cont2 == 7:
            !tar -zcf siete.tar.gz /kaggle/working/7_test/
            !rm -R 7_test
        if cont2 == 8:
            !tar -zcf ocho.tar.gz /kaggle/working/8_test/
            !rm -R 8_test
        if cont2 == 9:
            !tar -zcf nueve.tar.gz /kaggle/working/9_test/
            !rm -R 9_test
        cont2 += 1
        os.mkdir(str(cont2)+"_test")
        print(str(100*cont/len(unconnected_vertex_pairs))+'%')
if cont2 == 10:
    !tar -zcf diez.tar.gz /kaggle/working/10_test/
    !rm -R 10_test

lista=np.array(lista)
np.save('targets_test',lista)
print("listo")

Generando datos de entrenamiento
0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
tar: Removing leading `/' from member names
listo
Generando datos de test
tar: Removing leading `/' from member names
0.0%
tar: Removing leading `/' from member names
10.0%
tar: Removing leading `/' from member names
20.0%
tar: Removing leading `/' from member names
30.0%
tar: Removing leading `/' from member names
40.0%
tar: Removing leading `/' from member names
50.0%
tar: Removing leading `/' from member names
60.0%
tar: Removing leading `/' from member names
70.0%
tar: Removing leading `/' from member names
80.0%
tar: Removing leading `/' from member names
90.0%
tar: Removing leading `/' from member names
listo
